<a href="https://colab.research.google.com/github/yiiyama/qc-workbook-lecturenotes/blob/branch-2024/assignment_2024_05_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 【課題】量子フーリエ変換と量子ダイナミクスシミュレーション

In [ ]:
import sys
import shutil
import tarfile
from google.colab import drive
drive.mount('/content/gdrive')
shutil.copy('/content/gdrive/MyDrive/qcintro.tar.gz', '.')
with tarfile.open('qcintro.tar.gz', 'r:gz') as tar:
    tar.extractall(path='/root/.local')

sys.path.append('/root/.local/lib/python3.10/site-packages')

!git clone -b branch-2024 https://github.com/UTokyo-ICEPP/qc-workbook-lecturenotes
!cp -r qc-workbook-lecturenotes/qc_workbook /root/.local/lib/python3.10/site-packages/

In [ ]:
# まずは全てインポート
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Math
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.providers import JobStatus
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as AerSampler
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as RuntimeSampler
from qiskit_ibm_runtime.accounts import AccountNotFoundError
from qc_workbook.show_state import statevector_expr
from qc_workbook.optimized_additions import optimized_additions
from qc_workbook.utils import operational_backend, find_best_chain
from qc_workbook.dynamics import make_heisenberg_circuits, plot_heisenberg_spins

## 問題1: cos型の確率分布を作る

測定でビット列 $k$ を得る確率が $\frac{1}{2}[1+\cos(8\pi k /2^5)]$ となるような量子回路を作ってください。

In [ ]:
num_qubits = 5

circuit = QuantumCircuit(num_qubits)

##################
### EDIT BELOW ###
##################

# Set up a superposition of computational basis states

##################
### EDIT ABOVE ###
##################

# 実習の量子フーリエ変換回路

for itarg in range(num_qubits - 1, -1, -1):
    # 標的ビットにアダマールゲートをかける
    circuit.h(itarg)
    # target - 1から0まで制御ビットについてループ
    for ictrl in range(itarg - 1, -1, -1):
        # 標的と制御ビットのインデックスに応じた角度で制御Pゲートをかける
        power = ictrl - itarg - 1 + num_qubits
        circuit.cp((2 ** power) * 2. * np.pi / (2 ** num_qubits), ictrl, itarg)

    # 回路図を見やすくするためにバリアを入れる
    circuit.barrier()

# 最後にビットの順番を反転させる
for i in range(num_qubits // 2):
    circuit.swap(i, num_qubits - 1 - i)

circuit.measure_all()

# シミュレータ上のSamplerを利用
simulator = AerSimulator()
sampler = Sampler()
shots = 100000

circuit = transpile(circuit, backend=simulator)
sim_job = sampler.run([circuit], shots=shots)
counts_dict = sim_job.result()[0].data.meas.get_counts()

# 測定結果をプロットしやすいようにアレイに変換
counts = np.zeros(2 ** num_qubits)
for key, value in counts_dict.items():
    counts[int(key, 2)] = value
counts /= shots

# 測定結果と理論曲線をプロット
plt.scatter(np.arange(2 ** num_qubits), counts, label='observed')
x = np.linspace(0., 2 ** num_qubits, 400)
y = (1. + np.cos(8. * np.pi * x / 2 ** num_qubits)) / 2 ** num_qubits
plt.plot(x, y, label='target')
plt.legend();

## 問題2: ハイゼンベルグモデル、X方向のスピン

講義で扱ったハイゼンベルグモデルのシミュレーションで、各スピンの$X$方向の期待値の時間発展を測定する回路を書き、実習時と同様に時間に対してプロットしてください。

In [ ]:
n = 5
M = 10
omegadt = 0.1

shots = 100000

# Define the circuits
circuits = []

circuit = QuantumCircuit(n)

# Bit 0 in state 1/sqrt(2)(|0> + |1>)
circuit.h(0)

for istep in range(M):
    for j in range(n - 1):
        # ZZ
        circuit.cx(j, j + 1)
        circuit.rz(-omegadt, j + 1)
        circuit.cx(j, j + 1)

        # XX
        circuit.h(j)
        circuit.h(j + 1)
        circuit.cx(j, j + 1)
        circuit.rz(-omegadt, j + 1)
        circuit.cx(j, j + 1)
        circuit.h(j)
        circuit.h(j + 1)

        # YY
        circuit.p(-np.pi / 2., j)
        circuit.p(-np.pi / 2., j + 1)
        circuit.h(j)
        circuit.h(j + 1)
        circuit.cx(j, j + 1)
        circuit.rz(-omegadt, j + 1)
        circuit.cx(j, j + 1)
        circuit.h(j)
        circuit.h(j + 1)
        circuit.p(np.pi / 2., j)
        circuit.p(np.pi / 2., j + 1)

    # Copy of the circuit up to this point
    snapshot = circuit.copy()

    ##################
    ### EDIT BELOW ###
    ##################

    # Set up the observable for this snapshot
    #snapshot.?

    ##################
    ### EDIT ABOVE ###
    ##################

    snapshot.measure_all()
    circuits.append(snapshot)

simulator = AerSimulator()

circuits = transpile(circuits, backend=simulator)
sampler = Sampler()
sim_job = sampler.run(circuits, shots=shots)
sim_counts_list = [result.data.meas.get_counts() for result in sim_job.result()]

# Initial state as a statevector
initial_state = np.zeros(2 ** n, dtype=np.complex128)
initial_state[0:2] = np.sqrt(0.5)

plot_heisenberg_spins(sim_counts_list, n, initial_state, omegadt, add_theory_curve=True, spin_component='x')

**提出するもの**

- 問題1、2の完成した回路のコードとシミュレーション結果によるプロット
- 問題2で一般の方向のスピンの期待値を測定するためにはどうすればいいかの説明